In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [ ]:
tweet1=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
tweet2=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
print(tweet2.shape)

In [ ]:
tweet=pd.concat([tweet1,tweet2])

In [ ]:
tweet1.head()

In [ ]:
def remove(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


In [ ]:
tweet['text']=tweet['text'].apply(lambda x: remove(x))

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
#print(remove_html(example))

In [ ]:
tweet['text']=tweet['text'].apply(lambda x:remove_html(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)



In [ ]:
tweet['text']=tweet['text'].apply(lambda x:remove_emoji(x))

In [ ]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
tweet['text']=tweet['text'].apply(lambda x:remove_punct(x))

In [ ]:
!unzip 

In [ ]:
embedding_dict={}
with open('/kaggle/input/glove1/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
def create_corpus(tweet):
    corpus=[]
    for tweet in tqdm(tweet['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus(tweet)

In [ ]:
max_len=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)
tweet_pad=pad_sequences(sequences,maxlen=max_len,truncating='post',padding='post')


In [ ]:
print(tweet_pad.shape)

In [ ]:
print(tweet.shape[0])

In [ ]:
word_index=tokenizer_obj.word_index

In [ ]:
print(len(word_index))

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))
for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

In [ ]:
    from keras.models import Sequential
    from keras.layers import Dense, Activation, Embedding, LSTM
    import keras 
    embedding_vector_length = 100
    model = Sequential()
    model.add(Embedding(num_words,embedding_vector_length, input_length=max_len))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())

In [ ]:
model=Sequential()
embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),input_length=max_len)
model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(256,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
optimizer=Adam(learning_rate=1e-5)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])


In [ ]:
print(tweet.shape[0])

In [ ]:
train=tweet_pad[:tweet1.shape[0]]
test=tweet_pad[tweet1.shape[0]:]


In [ ]:
print(train.shape)

In [ ]:
print(test.shape)

In [ ]:
print(tweet.shape)

In [ ]:
tweet['target'].head()

In [ ]:
print(train.shape)

In [ ]:
print(tweet1.shape)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,tweet1['target'].values,test_size=0.20)
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
mod=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

In [ ]:
sub=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')


In [ ]:
y_predict=model.predict(test)
sub.target=y_predict
sub.to_csv("submission.csv",index=False)
